# Initialising the ID Verification Agent

In this notebook you will register the DID of the ID Verification agent on the Sovrin StagingNet and write a credential definition for the predefined Passport Details credential schema with ID - [2yWoMSfeTtYBFpUJFcnpqL:2:Passport Details:0.0.1](https://indyscan.io/tx/SOVRIN_STAGINGNET/domain/141253).

Schema : ```{
       "attrNames":[
           "PSS Passport No",
           "PSS Surname",
           "PSS Given Names",
           "PSS Nationality",
           "PSS Date of Birth",
           "PSS Sex",
           "PSS Place of Birth",
           "PSS Date of Issue",
           "PSS IssuingAuthority",
           "PSS Date Of Expiry",
           "PSS Photograph",
           "PSS Signature"
       ],
       "name":"Passport Details",
       "version":"0.0.1"
}
```


## 1. Initialise a controller for ID Verification Service Agent

In [1]:
%autoawait
import time
import asyncio
from aries_basic_controller.aries_controller import AriesAgentController
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_BASE = ""

WEBHOOK_PORT = 8052
ADMIN_URL = "http://id-verifier-agent:8051"

# Based on the aca-py agent you wish to control
agent_controller = AriesAgentController(webhook_host=WEBHOOK_HOST, webhook_port=WEBHOOK_PORT,
                                       webhook_base=WEBHOOK_BASE, admin_url=ADMIN_URL)

IPython autoawait is `on`, and set to use `asyncio`


## 2. Get current public DID

Before being able to write to any indy based ledger, your agent must have a public DID written on the ledger giving it the authority to write to it. As the cell below shows, this agent does not currently have a public DID. So any writes to the ledger will be rejected.

In [2]:
response = await agent_controller.wallet.get_public_did()
print(response)

{'result': None}


## 3. Generate a new DID

Before being able to write a DID to the ledger, you must create one using the wallet api. This api returns the identifier (the DID), and the verification key for that DID. A representation of it's public key. 

In [3]:
# generate new DID
response = await agent_controller.wallet.create_did()

did_object = response['result']
print("New DID", did_object)

New DID {'did': 'SQdUFDsVbJwY2vm3kXkDff', 'verkey': 'ErBK3YvuTinKn1pC5RGXax3ctM6rNAypW5WfwTT9ee1G', 'public': False}


## 4. Write DID to Sovrin Stagingnet

Anoyone can write a DID to the Sovrin StagingNet, it is a permissionless ledger. 

Visit [Sovrin Selfserve Portal](https://selfserve.sovrin.org) for more information. We have provided an automated process to write DIDs to Stagingnet in the step below.

In [4]:
# write new DID to Sovrin Stagingnet
import requests
import json 

url = 'https://selfserve.sovrin.org/nym'

payload = {"network":"stagingnet","did": did_object["did"],"verkey":did_object["verkey"],"paymentaddr":""}

# Adding empty header as parameters are being sent in payload
headers = {}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())
print(r.status_code)

{'statusCode': 200, 'headers': {'Access-Control-Allow-Origin': '*'}, 'body': '{"statusCode": 200, "SQdUFDsVbJwY2vm3kXkDff": {"status": "Success", "statusCode": 200, "reason": "Successfully wrote NYM identified by SQdUFDsVbJwY2vm3kXkDff to the ledger with role ENDORSER"}}'}
200


## 5. Accepting the Transaction Author Agreement (TAA)

Although the Sovrin StagingNet is permissionless, before DID's have the authority to write to the ledger they must accept something called a transaction author agreement by signing it using the DID they have on the ledger.

As a global public ledger, the Sovrin Ledger and all its participants are subject to privacy and data protection regulations such as the EU General Data Protection Regulation (GDPR). These regulations require that the participants be explicit about responsibilities for Personal Data.

To clarify these responsibilities and provide protection for all parties, the Sovrin Governance Framework Working Group developed an agreement between Transaction Authors and the Sovrin Foundation. The TAA can be found at Sovrin.org. It ensures that users are aware of and consent to the fact that all data written to the Sovrin Ledger cannot be removed, even if the original author of the transaction requests its removal.

The TAA outlines the policies that users must follow when interacting with the Sovrin Ledger. When a user’s client software is preparing a transaction for submission to the network, it must include a demonstration that the user had the opportunity to review the current TAA and accept it. This is done by including some additional fields in the ledger write transaction: 

* A hash of the agreement
* A date when the agreement was accepted, and
* A string indicating the user interaction that was followed to obtain the acceptance.

The Indy client API used by Sovrin has been extended to allow users to review current and past agreements and to indicate acceptance through an approved user interaction pattern. - source: https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/

For more details on TAA please read more at the following links:
* [Preparing for the Sovrin Transaction Author Agreement](https://sovrin.org/preparing-for-the-sovrin-transaction-author-agreement/)
* [How the recent approval of the Sovrin Governance Framework v2 affects Transaction Authors
](https://sovrin.org/how-the-recent-approval-of-the-sovrin-governance-framework-v2-affects-transaction-authors/)
* [TAA v2](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/TAA.md)
* [TAA Acceptance Mechanism List (AML)](https://github.com/sovrin-foundation/sovrin/blob/master/TAA/AML.md)

In [5]:
response = await agent_controller.ledger.get_taa()
TAA_text = response['result']['taa_record']['text']
TAA_version = response['result']['taa_record']['version']
TAA_digest = response['result']['taa_record']['digest']


taa_record = {
            "version": TAA_version,
            "mechanism": "service_agreement",
            'text': TAA_text
}

TAA_to_sign = json.dumps(taa_record)

In [6]:
response = await agent_controller.ledger.accept_taa(TAA_to_sign)
## Will return {} if successful
print(response)

{}


## 6. Set public DID

Now you are able to assign the DID written to the ledger as public.

In [7]:
response = await agent_controller.wallet.assign_public_did(did_object["did"])
print(response)

{'result': {'did': 'SQdUFDsVbJwY2vm3kXkDff', 'verkey': 'ErBK3YvuTinKn1pC5RGXax3ctM6rNAypW5WfwTT9ee1G', 'public': True}}


## 7. Get public DID

In [8]:
response = await agent_controller.wallet.get_public_did()
print(response)
issuer_nym = response['result']['did']
print('\nIssuer public DID:',issuer_nym)

{'result': {'did': 'SQdUFDsVbJwY2vm3kXkDff', 'verkey': 'ErBK3YvuTinKn1pC5RGXax3ctM6rNAypW5WfwTT9ee1G', 'public': True}}

Issuer public DID: SQdUFDsVbJwY2vm3kXkDff


## 8. Fetch verkey for public DID

Additionally, we can verify that this DID does actually resolve to the public key material on the ledger.

In [9]:
issuer_verkey = await agent_controller.ledger.get_did_verkey(issuer_nym)
print(issuer_verkey)

{'verkey': 'ErBK3YvuTinKn1pC5RGXax3ctM6rNAypW5WfwTT9ee1G'}


## 9. Get public DID endpoint

As well as providing a publically accessible endpoint to contact the DID controller through the agent framework.

In [10]:
issuer_endpoint = await agent_controller.ledger.get_did_endpoint(issuer_nym)
print(issuer_endpoint)

{'endpoint': 'https://5b5f1e11be77.ngrok.io'}


## 10. Write Credential Definition for Passport Details to Ledger

The ID Verification Agent must have a credential definition on the ledger to issue these credentials.



In [11]:
schema_id = "2yWoMSfeTtYBFpUJFcnpqL:2:Passport Details:0.0.1"
response = await agent_controller.definitions.write_cred_def(schema_id)

cred_def_id = response["credential_definition_id"]
print(cred_def_id)

%store cred_def_id
%store schema_id

SQdUFDsVbJwY2vm3kXkDff:3:CL:141253:default
Stored 'cred_def_id' (str)
Stored 'schema_id' (str)


## 11. End of Tutorial

Be sure to terminate the controller so you can run another tutorial.

In [12]:
response = await agent_controller.terminate()
print(response)

None


# Proceed to [Part 1.3 - Issue Passport Details Credential](http://localhost:8888/notebooks/Part%201.3%20-%20Issue%20Passport%20Details%20Credential.ipynb) to issue a Passport Details credential to your Doctor wallet.